# Ejercicio 1

In [ ]:
# Importar módulos
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('cotizacion.csv', delimiter=';')

In [ ]:
df.info()

In [ ]:
df

In [ ]:
df['Volumen']=df['Volumen'].astype('str')

In [ ]:
df[['Vol1','Vol2']]=df['Volumen'].str.rsplit('.',n=1, expand=True)

In [ ]:
df['Vol1'] = df['Vol1'].str.replace('.', '', regex=False)

In [ ]:
df['Volumen'] = df['Vol1'].str.cat(df['Vol2'], sep='.')

In [ ]:
df.head(3)

In [ ]:
for x in df[['Final','Máximo','Mínimo','Efectivo']]:
    df[x] = df[x].str.replace(',', '.').astype(float)

In [ ]:
df['Volumen']=df['Volumen'].astype('float')

In [ ]:
df.info()

In [ ]:
df=df.iloc[:,np.r_[0:6]]

In [ ]:
df.head(3)

In [ ]:
def resumen(df):
    columnas_numericas = df.select_dtypes(include=['number']).columns
    nombres = []
    minimo = []
    maximo = []
    media = []

    for x in columnas_numericas:
        nombres.append(x)
        minimo.append(df[x].min())
        maximo.append(df[x].max())
        media.append(df[x].mean())

    df_resumen = pd.DataFrame({
        'Variable': nombres,
        'Mínimo': minimo,
        'Máximo': maximo,
        'Media': media
    })
    
    return df_resumen

In [ ]:
resumen(df)

# Ejercicio 2

## 1

In [ ]:
# Importa df 
df=pd.read_csv('titanic.csv')


## 2

In [ ]:
df.info()

In [ ]:
df.head(10)

In [ ]:
df.tail(10)

## 3

In [ ]:
df[df['PassengerId']==148]

## 4

In [ ]:
df.iloc[::2]

## 5

In [ ]:
df[df['Pclass']==1].sort_values('Name')

## 6

In [ ]:
(df.Survived.value_counts() / len(df))*100

## 7

In [ ]:
survived=df[df['Survived']==1]

In [ ]:
round(pd.crosstab(survived['Pclass'],survived['Survived'], normalize='columns')*100, 2)

## 8

In [ ]:
df['Age'].isnull().value_counts()

In [ ]:
df.dropna(subset='Age',inplace=True)

In [ ]:
df['Age'].isnull().value_counts()

## 9

In [ ]:
df[df['Sex']=='female'].groupby(['Pclass','Sex'])[['Age']].mean()

## 10

In [ ]:
def legal_age(x):
    if x > 18:
        return True
    else:
        return False

In [ ]:
df['legal_age'] = df['Age'].apply(legal_age)


In [ ]:
df.tail(20)

## 11

In [ ]:
pd.crosstab(df['Pclass'],df['legal_age'],normalize=True)*100

# Ejercicio 3

## 1

In [ ]:
emisiones2016=pd.read_csv('emisiones-2016.csv', delimiter=';')
emisiones2017=pd.read_csv('emisiones-2017.csv', delimiter=';')
emisiones2018=pd.read_csv('emisiones-2018.csv', delimiter=';')
emisiones2019=pd.read_csv('emisiones-2019.csv', delimiter=';')

In [ ]:
print(emisiones2016.info())
print(emisiones2017.info())
print(emisiones2018.info())
print(emisiones2019.info())

In [ ]:
df = pd.concat([emisiones2016, emisiones2017, emisiones2018, emisiones2019], ignore_index=True)

In [ ]:
df.info()

## 2

In [ ]:
df=df.filter(regex='^ESTACION$|^MAGNITUD$|^ANO$|MES$|^D')

In [ ]:
df.info()

## 3

In [ ]:
df=pd.melt(df, id_vars=df.iloc[:,np.r_[0:4]],value_vars=df.iloc[:,np.r_[4:35]],var_name='DIA', value_name='CONTAMINANTES')

In [ ]:
df.head(3)
df.tail(3)


## 4

In [ ]:
df['DIA']=df['DIA'].str.replace('D','')

In [ ]:
df.info()

In [ ]:
df['FECHA']=df["ANO"].astype(str) + "/" + df["MES"].astype(str) + "/" + df["DIA"]

In [ ]:
df['FECHA']=pd.to_datetime(df['FECHA'], errors='coerce')

In [ ]:
df

## 5

In [ ]:
np.isnat(df['FECHA']).value_counts()

In [ ]:
df=df[~np.isnat(df['FECHA'])]

In [ ]:
np.isnat(df['FECHA']).value_counts()

In [ ]:
df=df.sort_values(by=['ESTACION', 'FECHA'])

## 6

In [ ]:
print('Los valores de estación son: ',df['ESTACION'].unique())
print('Los valores de magnitud son: ',df['MAGNITUD'].unique())

## 7

In [ ]:
def obtener_emisiones(data, estacion, magnitud, fecha_inicio, fecha_fin, 
                        col_estacion='ESTACION', col_fecha='FECHA',col_magnitud='MAGNITUD', col_emisiones='CONTAMINANTES'):
  
    estaciones_validas = data[col_estacion].unique()
    magnitudes_validas = data[col_magnitud].unique()

    if estacion not in estaciones_validas:
        raise ValueError(f"Estación '{estacion}' no encontrada. Estaciones válidas: {', '.join(map(str, estaciones_validas))}")
    
    if magnitud not in magnitudes_validas:
        raise ValueError(f"Magnitud '{magnitud}' no encontrada. Magnitudes válidas: {', '.join(map(str, magnitudes_validas))}")
    
    try:
        fecha_inicio_dt = pd.to_datetime(fecha_inicio)
        fecha_fin_dt = pd.to_datetime(fecha_fin)
    except ValueError as e:
        raise ValueError(f"Formato de fecha inválido. Use 'YYYY-MM-DD'. Error: {str(e)}")
    
    if fecha_inicio_dt > fecha_fin_dt:
        raise ValueError(f"Fecha de inicio {fecha_inicio} debe ser anterior o igual a fecha fin {fecha_fin}")
    
    data[col_fecha] = pd.to_datetime(data[col_fecha])
    min_fecha = data[col_fecha].min()
    max_fecha = data[col_fecha].max()
    
    if fecha_inicio_dt > max_fecha or fecha_inicio_dt < min_fecha or fecha_fin_dt > max_fecha or fecha_fin_dt < min_fecha:
        raise ValueError(f"Rango de fechas fuera de los datos disponibles. Rango en datos: {min_fecha.date()} a {max_fecha.date()}")
    
    filtro = df[(data[col_estacion]==estacion)&(data[col_magnitud]==magnitud)&(data[col_fecha]>=fecha_inicio_dt)&
                (data['FECHA']<=fecha_fin_dt)]
    filtro.reset_index(drop=True, inplace=True)
    filtro=filtro[[col_estacion,col_magnitud,col_fecha,col_emisiones]] 
    filtro=filtro.rename(columns={col_emisiones:'EMISIONES'})
    return filtro


In [ ]:
obtener_emisiones(df, 4, 7, '2016-01-10', '2016-01-20', col_estacion='ESTACION', col_magnitud='MAGNITUD', col_fecha='FECHA', col_emisiones='CONTAMINANTES')

In [ ]:
# def resumen_emisiones_agrupadas(datos, col_magnitud='MAGNITUD', col_emision='EMISION'):
#     import pandas as pd
#     import numpy as np
#     from scipy.stats import kurtosis

    
#     columnas_requeridas = {col_magnitud, col_emision}
#     if not columnas_requeridas.issubset(datos.columns):
#         faltantes = columnas_requeridas - set(datos.columns)
#         raise ValueError(f"Columnas faltantes en los datos: {', '.join(faltantes)}")
    
#     if not pd.api.types.is_numeric_dtype(datos[col_emision]):
#         raise ValueError(f"La columna '{col_emision}' debe contener valores numéricos")
    
#     def cv(x):
#         return np.std(x, ddof=1) / np.mean(x) * 100 if np.mean(x) != 0 else np.nan
    
#     def kurt(x):
#         return kurtosis(x, fisher=True, bias=False)
    
#     resumen = datos.groupby(col_magnitud)[col_emision].agg(
#         Muestras='count',
#         Mínimo='min',
#         Máximo='max',
#         Media='mean',
#         Mediana='median',
#         Desv_Estándar='std',
#         Varianza='var',
#         Sesgo='skew',
#         Curtosis=kurt,
#         Percentil_25=lambda x: x.quantile(0.25),
#         Percentil_75=lambda x: x.quantile(0.75),
#         IQR=lambda x: x.quantile(0.75) - x.quantile(0.25),
#         Coef_Variación=cv 
#     )
    
#     # Reordenar columnas para mejor presentación
#     column_order = [
#         'Muestras', 'Mínimo', 'Percentil_25', 'Mediana', 'Media', 
#         'Percentil_75', 'Máximo', 'IQR', 'Desv_Estándar', 
#         'Varianza', 'Coef_Variación', 'Sesgo','Curtosis'
#     ]
    
#     return resumen[column_order].round(2)

In [ ]:
# resumen_emisiones_agrupadas(df, 'MAGNITUD', 'CONTAMINANTES')

## 10

In [ ]:
def resumenDescriptivo(estacion, contaminante):
    return df[(df.ESTACION == estacion) & (df.MAGNITUD == contaminante)].CONTAMINANTES.describe()

estacion=4
contaminante=6
print(f"Resumen descriptivo para estación {estacion} y contaminante {contaminante}:\n{resumenDescriptivo(estacion, contaminante)}")
estacion=60
contaminante=14
print(f"Resumen descriptivo para estación {estacion} y contaminante {contaminante}:\n{resumenDescriptivo(estacion, contaminante)}")
